In [ ]:
#EPOS-OC GFZ IMPLEMENTATION 
# initially, no previous phase observation ...
t1=None
    
# to calculate this we use the nominal frequencies fn_2 and fn_4
k_diob = 32    #shift factor for DIOB beacon
    
feN2, feN4 = beacon_nominal_frequency(k_diob)
print(f"feN2 (2 GHz channel) = {feN2:.9f} Hz")
print(f"feN4 (400 MHz channel) = {feN4:.9f} Hz")

feN3 = ((feN2**2 - feN4**2) / (feN2 - feN4))
sf = (feN2 / feN4)**2

print(f"feN3 (2 GHz channel) = {feN3:.9f} Hz")
lamda_3 = 299792458.0 / feN3
# store results here 
t = []; res_gop=[];

# for every block in the RINEX file
for block in rnx:
    # for every beacon in the block
    for beacon, data in block:
        # match DIOB
        if beacon == diob:
            # use the block, RINEX provided clock correction to get to (approximate) TAI
            # date TAI = epoch + receiver clock offset
            t2 = block.t() + attotime.attotimedelta(nanoseconds=block.clock_offset()*1e9)
            # satellite position at the current epoch (taking into account signal travel time)
            satx, saty, satz, _, _ = alg.sat_at_emission_time(diob_xyz, t2, intrp, None)
            # L1 and L2 measurements
            L2_1 = data['L1']['value']
            L2_2 = data['L2']['value']
            #L3 build
            L3 = (sf*L2_1 - math.sqrt(sf)*L2_2)/(sf-1)
            # beacon-to-satellite distance
            r2 = np.linalg.norm(np.array([satx, saty, satz])-np.array(diob_xyz))
            # if we have a previous measurement, within 10 sec ...
            if t1 is not None and (t2 - t1).total_seconds() <= 10.:
                # computed Doppler cycles
                dccom = (r2 - r1) / lamda_3
                print(f"dcmes = {dcmes:.3f}")
                # observed Doppler cycles
                dcobs = float((t2-t1).total_nanoseconds()) * ()
                print(f"dcobs = {dcobs:.3f}")
                # very loose outlier detection
                if abs(dccom - dcmes) > 100000.:
                    pass
                else:
                    t.append(at2pt(t2))
                    res_gop.append(dccom - dcmes)
            # prepare for next measurement
            r1 = r2
            t1 = t2
            L1 = L3